# Segmentação e criação do vetor de características

**Observação inicial**: para os trabalhos posteriores, será levado em consideração que os dados estão corretos e filtrados conforme na aula de "Pré-processamento". Portando, antes de iniciar este conteúdo, vocês terão que ter concluído as questões passadas e como **tarefa** salvar um arquivo com dados filtrados (não executar todo `preprocessing.ipynb`).

Serão utilizados nesta aula os dados sem nenhuma filtragem e com o conjunto de canais escolhidos aleatoriamente.

## Introdução

Um formato importante do *dataset* para a classificação dos dados, é estar organizado preferencialmente em duas dimensões. As linhas serão as amostras (rotuladas ou não) e as colunas, as características. Além disso, os dados para cada uma das características deve fazer algum sentido para a boa atuação do classificador. Para essa matriz final, damos o nome de `vetor de características`.

Em experimentos SSVEP-BCI, a característica mais forte é o `PSD` (*Power Spectral Density*). O `PSD`, como o nome sugere, é obtido por meio do sinal no domínio da frequência, aplicando a seguinte fórmula: $|x_i|^2$. O `PSD` potencializa a energia das frequências mais evidentes, melhorando o desempenho de classificação.

Alguns métodos da biblioteca MNE nos dão um vetor de características pronto. Porém, é interessante realizarmos algumas etapas passo a passo sem o uso inicial da biblioteca para entendermos o funcionamento do método e alterar como quisermos.


## Transformação de domínio (e segmentação)

O `shape` inicial dos dados é: `(125, 256, 1205) -> (trials, channels, data)`. Vamos aplicar a Transformada Rápida de Fourier em Tempo Curto (STFT) (após carregar e filtrar os dados):

In [1]:
import matplotlib
import mne
from scipy.signal import stft

%matplotlib inline

In [2]:
# carregamento do dataset (FIF file)
epochs = mne.read_epochs('files/ssvep-epo.fif')
# filtranndo apenas alguns canais
epochs.pick_channels(['E108', 'E109', 'E116', 'E125', 'E118', 'E117', 'E126',
                      'E139', 'E127', 'E138', 'E140', 'E150', 'E151'])
print(epochs)

Reading files/ssvep-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    4995.85 ms
        0 CTF compensation matrices available
125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
<EpochsFIF  |   125 events (all good), 0 - 4.99585 sec, baseline off, ~15.1 MB, data loaded,
 '1': 25
 '2': 25
 '3': 30
 '4': 25
 '5': 20>


In [24]:
# extraindo somente os dados do objeto MNE
data = epochs.get_data()
print(data.shape) # domínio do tempo

# aplicando STFT
_, _, w = stft(data, fs=241, nperseg=32, noverlap=16)
# w = np.swapaxes(w, 3, 4)
print(w.shape)

(125, 13, 1205)
(125, 13, 17, 77)


Obemos um `shape` diferente, acrescentando uma dimensão a mais em nossos dados. Isso é devido a quantidade de janelas ou segmentos informados (`nperseg`) e a sobreposição utilizada (`overlap`). **DISCUSSÃO EM AULA**

Aplicando o `PSD` teremos:

In [31]:
w = np.abs(w) ** 2
# w = np.reshape(w, (125, 13, 17 * 77)) # <= questão de projeto
# w = w.transpose(0, 2, 1)
# w = np.reshape(w, (125 * 1309, 13))
print(w.shape)

# shape resultante: (125, 13, 17, 77)

(125, 13, 17, 77)


## Extração de características

Não é uma boa estratégia utilizar os dados "crus" de `PSD` para a classificação. Desta forma, vamos adotar alguns algoritmos simples para reduzir uma dimensão dos dados e potencializar nossas características. Uma lista de característica é listada [por este artigo intitulado "*A wearable wireless brain-computer interface using steady-state visual evoked potentials*"](https://www.researchgate.net/publication/334854837_A_wearable_wireless_brain-computer_interface_using_steady-state_visual_evoked_potentials). Já que temos o PSD dos dados, vamos demonstrar a aplicação do "*Mean of PSD*" ou `FMN`:

In [32]:
import numpy as np

fmn = np.mean(w, axis=-1)
print('FMN:', fmn.shape)

# Root of sum of squares
rss = np.sqrt(np.sum(w ** 2, axis=-1))
print('RSS:', rss.shape)

FMN: (125, 13, 17)
RSS: (125, 13, 17)


Após a aplicação de algumas características, juntamos todas elas no mesmo conjunto de dados e transformamos cada eletrodo em uma característica. Em outras palavras, o *shape* final que ficou no seguinte formato:`(125, 13, 17)`. Agora deverá ficar `(125 * 17, 13) => (2125, 13)`.

Se mais características fossem adicionadas, elas entrariam como multiplicação nas colunas. No exemplo anterior temos apenas uma característica desenvolvida. Se adicionarmos 4 características, o `shape` do vetor de características ficaria no seguinte formato: `(2125, 13 * 4) => (2125, 52)`. Explicando os dados, seriam 2125 amostras e 52 características.

In [33]:
# realização das transformações finais (TAREFA)


## Questões de projeto

1) Nem sempre os canais são vistos como características. Uma outra forma é adicionar os canais às amostras (reduzindo a quantidade de características e aumentando a quantidade de amostras). O resultado disso deve ser avaliado.

2) É comum a aplicação de algum algoritmo para reduzir todos os canais ou transformar apenas em um (que é o caso de aplicar a média de todos os eletrodos/canais).

3) Adicionar características ruins confundem o resultado? Características que não estão relacionadas ao domínio do problema pode ser ruim? Isso deve ser avaliado...